# 데이터 정제

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_excel("/content/drive/Shareddrives/NLP모델링/코드 연습/예진/강의평_eng.xlsx")

In [4]:
df= df.rename(columns={'Unnamed: 9':'eng'})

In [5]:
import re
df['eng'] = df['eng'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

In [6]:
dfl =df
dfl=dfl[['학정번호', '강의명/교수명',
       '강의명', '교수명', '유의사항', '별점', '수강시기', '강의평', 'eng']]

## 비지도학습 기반 감성 분석(lexicon을 활용한 감성분석)

## sentiwordnet 이용

In [ ]:
import nltk
nltk.download('all')

In [ ]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import sentiwordnet as swn

In [ ]:
# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return 

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

In [ ]:
def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    li=[] #감성을 유발한 lemma(어근)담기
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            li.append(lemma)
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1,li
    
    return 0,li

## vader이용

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(dfl['eng'][1])
print(senti_scores)
#the compound score is the sum of positive, negative & neutral scores
# which is then normalized between -1(most extreme negative) and +1 (most extreme positive).

# threshold 기준
threshold_l, threshold_u   
 0: negative 1: positive 0.5: neutral  

여러 번의 실험결과 0,0.1이 최적에 가까웠다.

In [ ]:
def vader_polarity(review,threshold_l = 0.0, threshold_u = 0.1 ):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    agg_score = scores['compound']
    if agg_score >= threshold_u:
      final_sentiment = 1
    elif agg_score <= threshold_l:
       final_sentiment = 0
    else:
       final_sentiment = 0.5
    return final_sentiment

In [ ]:
dfl['vader_preds'] = dfl['eng'].apply( lambda x : vader_polarity(x, 0, 0.1) )

# word2vec 임베딩 : 성적, 로드, 강의력, 교수성향에 대한 각각의 라벨링을 위해 각 주제와 관련이 있는 단어를 추출하자.

1. sentiwordnet에서 뽑힌 모든 단어를 unique하게 리스트업하고, 빈도수 계산 : trigger(counter)
2. 영어강의평 word2vec 훈련
3. 성적, 로드, 강의력, 교수성향과 관련된 단어 추출
4. trigger list들에 대해 stop word 제거하고 남은 단어들에 대해 3번과 일치하는 값이 존재하면 Y
5. Y로 선택된 값은 감성분석 결과를 해당 카테고리의 값으로 부여
6. 최종 점수 부여: 직접 라벨링한 것 0.3 각각을 라벨링한 것 0.7로 가중평균 구한 열 추가
(만약 직접 라벨링이 없으면 직접 라벨링한 값으로 대체하되, 평점이 1점이면 무조건 0, 평점이 5점이면 무조건 1로 부여)


In [ ]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


### 1. sentiwordnet에서 뽑힌 모든 단어를 unique하게 리스트업하고, 빈도수 계산 : trigger(counter)

In [ ]:
u=[] #trigger word가 없는 인덱스를 u에 담음
for i in range(len(dfl)):
  if type(dfl['sentiwordnet_preds'].iloc[i])!= tuple:
    u.append(i)

In [ ]:
for i in range(len(dfl)):
  if i not in u:
    dfl['sentiwordnet_points'].iloc[i] = dfl['sentiwordnet_preds'].iloc[i][0]
    dfl['sentiwordnet_words'].iloc[i] = dfl['sentiwordnet_preds'].iloc[i][1]

In [ ]:
all=[]
for i in range(len(dfl)):
  if i not in u:
    all.extend(dfl["sentiwordnet_words"].iloc[i])

In [ ]:
trigger = Counter(all)

In [ ]:
def preprocessing(text):
  content_text = re.sub(r'\([^)]*\)', '',text)
  # 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
  sent_text = sent_tokenize(content_text)
  # 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
  normalized_text = []
  for string in sent_text:
      tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
      normalized_text.append(tokens)
  # 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
  result = [word_tokenize(sentence) for sentence in normalized_text]
  return result

### 2.영어 강의평 word2vec 훈련

In [ ]:
dfl_w2v = dfl[['eng']]
dfl_w2v["token"] =0 

In [ ]:
dfl_w2v["token"] = (dfl_w2v["eng"]).apply( lambda x : preprocessing(x) )

In [ ]:
all_w2v=[]
for i in range(len(dfl_w2v)):
  if i not in u:
    all_w2v.extend(dfl_w2v["token"].iloc[i])

In [ ]:
model = Word2Vec(sentences=all_w2v, size=100, window=5, min_count=5, workers=4, sg=0)

### 3. 성적, 로드, 강의력, 교수성향과 관련된 단어 추출

In [ ]:
load = model.most_similar("load",topn=50)
quality = model.most_similar("quality", topn = 50)
scores = model.most_similar("scores",topn = 50)
professor = model.most_similar("professor")
teach = model.most_similar("teach")
teach_df = pd.DataFrame(teach, columns = ['professor_words', 'similarity' ])

In [ ]:
k= pd.DataFrame(professor, columns = ['professor_words', 'similarity' ])
b = pd.DataFrame(quality, columns = ['quality_words', 'similarity' ])
c = pd.DataFrame(load, columns = ['load_words', 'similarity' ])
d = pd.DataFrame(scores, columns = ['scores_words', 'similarity' ])

In [ ]:
l = pd.concat([k, teach_df], axis =0)
l.reset_index(drop=True, inplace=True)
trigger_df = pd.concat([l, b,c,d], axis = 1) 

In [ ]:
a1 = trigger_df[['professor_words']]
b1 = trigger_df[['load_words']]
c1 = trigger_df[['scores_words']]
d1 = trigger_df[['quality_words']]

In [ ]:
c1 = trigger_df[['scores_words']]

In [ ]:
a1.loc[50,'professor_words']='professor'

In [ ]:
load_plus = ['challenge', 'easy' ,'honey']
professor_plus =['professor', 'honey', 'difficult', 'assignment','report','honey','god','light']
scores_plus = ['quiz','exam','test', 'credit','score','grade']

In [ ]:
j=0
for i in scores_plus:
   c1.loc[50+j,'scores_words']= i 
   j+=1

In [ ]:
j=0
for i in load_plus:
   b1.loc[50+j,'load_words']= i 
   j+=1

In [ ]:
j=0
for i in professor_plus:
   a1.loc[50+j,'professor_words']= i 
   j+=1

In [ ]:
trigger_df['professor_words'] = trigger_df['professor_words'].append(pd.Series(professor_plus),ignore_index=True)

### 4. trigger list들에 대해 stop word 제거하고 남은 단어들에 대해 3번과 일치하는 값이 존재하면 Y

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 


result_ = []
for token in all: 
    if token not in stop_words: 
        result_.append(token) 

In [ ]:
result_ = []
for token in all: 
    if token not in stop_words: 
        result_.append(token) 

In [ ]:
trigger_sw_removed = Counter(result_)

In [ ]:
my_set  = set(result_)
trigger_pre = list(my_set)

In [ ]:
df_trigger_pre = pd.DataFrame(trigger_pre, columns= ["word"])
df_trigger_pre["similar"]=0
i = 0
for j in trigger_pre:
    try:
      df_trigger_pre["similar"].iloc[i] = model.most_similar(j, topn=20)
      i+=1

    except KeyError:
      print("a")
      i+=1

In [ ]:
dfl_d = dfl.drop(u)

In [ ]:
for i in dfl_d.index:

    if any(word in dfl_d.loc[i,'sentiwordnet_words'] for word in c1 ) : 
      dfl_d.loc[i,'성적'] = 'Y'
    if any(word in dfl_d.loc[i,'sentiwordnet_words'] for word in b1) : 
      dfl_d.loc[i,'로드'] = 'Y'
    if any(word in dfl_d.loc[i,'sentiwordnet_words'] for word in d1) : 
      dfl_d.loc[i,'강의력'] = 'Y'
    if any(word in dfl_d.loc[i,'sentiwordnet_words'] for word in a1) : 
      dfl_d.loc[i,'교수님성향'] = 'Y'

In [ ]:
for i in dfl_d.index:
    for word in dfl_d.loc[i,'sentiwordnet_words']:
      if word in c1:
          dfl_d.loc[i,'성적'] = 'Y'

### 5. Y로 선택된 값은 감성분석 결과를 해당 카테고리의 값으로 부여 

In [ ]:
for i in dfl_d.index:
  if dfl_d.loc[i,'성적'] == 'Y':     
      dfl_d.loc[i,'성적_점수'] = int(dfl_d.loc[i, "vader_preds"])
  if dfl_d.loc[i,'로드'] == 'Y':     
    dfl_d.loc[i,'로드_점수'] = int(dfl_d.loc[i, "vader_preds"])
  if dfl_d.loc[i,'강의력'] == 'Y':   
    dfl_d.loc[i,'강의력_점수'] = int(dfl_d.loc[i, "vader_preds"])
  if dfl_d.loc[i,'교수님성향'] == 'Y':    
    dfl_d.loc[i,'교수님성향_점수'] = int(dfl_d.loc[i, "vader_preds"])

In [ ]:
dfl_d.to_csv("/content/drive/Shareddrives/NLP모델링/코드 연습/예진/allinone.csv")

### 6. 최종 점수 부여

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/NLP모델링/코드 연습/예진/allinone.csv")

In [ ]:
df = df[['강의명/교수명', '강의평', 'eng', 'vader_preds',   'sentiwordnet_points', 'sentiwordnet_words']]

In [ ]:
score = df[['학정번호', '강의명/교수명', '별점', 'vader_preds',   'sentiwordnet_points', '강의력_점수', '교수님성향_점수', '로드_점수', '성적_점수']]

직접 라벨링한 것 0.3 각각을 라벨링한 것 0.7로 가중평균 구한 열 추가 (만약 직접 라벨링이 없으면 직접 라벨링한 값으로 대체하되, 평점이 1점이면 무조건 0, 평점이 5점이면 무조건 1로 부여

1. 성적

- 후하게 주면 1 후하지 않으면 0 

2. 로드 

- 쉬우면 1 어려우면 0

3. 강의력

- 좋으면 1 나쁘면 0

4. 교수님 성향

- 좋으면 1 나쁘면 0 

- 전부다 긍정이 1 부정이 0 중간이 0.5

- vader_preds(Y여서 라벨링된 ~_점수열) * 0.7 + hand * 0.3 = final if 
vader_preds(Y여서 라벨링된 ~_점수열)이 nan이면 hand로

- 족보여부와 강의자료질은 직접 라벨링한 결과를 따름

In [ ]:
hand = pd.read_csv("/content/drive/Shareddrives/NLP모델링/labeling - labeling (1).csv",encoding='cp949')

In [ ]:
hand = hand[['강의명/교수명', '성적 (너그러움 1, 보통 null, 깐깐함 0)',
       '로드 (쉬우면 1, 빡세면 0, 애매하면 null)', '강의력 (좋으면 1, 나쁘면 0)',
       '교수님성향 (좋으면 1, 안 좋으면 0)', '족보여부', '강의자료질']]

In [ ]:
hand = hand.fillna(0.5)

In [ ]:
merge = pd.merge(score,hand, how='inner',on='강의명/교수명')

In [ ]:
merge = merge.fillna(100) #nan값은 100으로 치환

In [ ]:
for i in range(len(merge)):
  if merge.loc[i,'로드_점수']!= 100 :
    merge.loc[i,'final_로드'] = merge.loc[i,'로드_점수']*0.7+ merge.loc[i,'로드 (쉬우면 1, 빡세면 0, 애매하면 null)']*0.3
  else:
    merge.loc[i,'final_로드'] = merge.loc[i,'로드 (쉬우면 1, 빡세면 0, 애매하면 null)']

In [ ]:
for i in range(len(merge)):
  if merge.loc[i,'성적_점수']!= 100 :
    merge.loc[i,'final_성적'] = merge.loc[i,'성적_점수']*0.7+ merge.loc[i,'성적 (너그러움 1, 보통 null, 깐깐함 0)']*0.3
  else:
    merge.loc[i,'final_성적'] = merge.loc[i,'성적 (너그러움 1, 보통 null, 깐깐함 0)']

In [ ]:
for i in range(len(merge)):
  if merge.loc[i,'교수님성향_점수']!= 100 :
    merge.loc[i,'final_교수'] = merge.loc[i,'교수님성향_점수']*0.7+ merge.loc[i,'성적 (너그러움 1, 보통 null, 깐깐함 0)']*0.3
  else:
    merge.loc[i,'final_교수'] = merge.loc[i,'교수님성향 (좋으면 1, 안 좋으면 0)']

In [ ]:
for i in range(len(merge)):
  if merge.loc[i,'강의력_점수']!= 100 :
    merge.loc[i,'final_강의력'] = merge.loc[i,'강의력_점수']*0.7+ merge.loc[i,'강의력 (좋으면 1, 나쁘면 0)']*0.3
  else:
    merge.loc[i,'final_강의력'] = merge.loc[i,'강의력 (좋으면 1, 나쁘면 0)']

In [ ]:
final_score = merge[['학정번호', '강의명/교수명', '별점', 'final_성적', 'final_교수',
       'final_로드', 'final_강의력','족보여부', '강의자료질']]

In [ ]:
final_score.to_csv("/content/drive/Shareddrives/NLP모델링/코드 연습/예진/final_score.csv")